# Bar charts

In this notebook, we'll deep dive on Holoviews Bars, and do somethings with pandas dataframes along the way.

In [ ]:
import pandas as pd
import holoviews as hv

hv.extension('bokeh')

The sample data we're looking at are the medal winners from the 2014 Olympics.

## Data extraction

The next cell fetches and displays the data. There's quite a lot of it: when you've finished looking, click on the blue bar to the left of the output cell to collapse the cell. (Move your mouse to the left of the output until the blue bar appears.)

In [ ]:
from bokeh.sampledata.olympics2014 import data
data

The data is in a dictionary with three top level keys. The interesting key is `data`, which contains a list of dictionaries. Fortunately, pandas knows how to intepret this.

In [ ]:
df = pd.DataFrame.from_records(data['data'])
df

The `name` and `abbr` columns are fine, but the `medals` column contains a list of medals in each row. We can easily extract the data in there to separate columns.

### Extract medals - row by row

One way to extract the medals is to use the `.apply()` method to extract the medals one column at a time. The `apply()` method applies a function row-by-row to the column to produce another column. In this case the function accesses the dictionary in the `medals` column.

In [ ]:
tmp_df = df.copy()
for medal in ['gold', 'total', 'silver', 'bronze']:
    tmp_df[medal] = tmp_df['medals'].apply(lambda m:m[medal])

tmp_df.head()

### Extract medals - via another dataframe

Using the `.apply()` method is slower than operating on entire column at a time. In fact, you should avoid using loops on dataframes if you can, in favour of using dataframe operations. (Sometimes it's unavoidable, because you want to do something that the datframe API can't handle. Sometimes it's just easier to use `.apply()`, and on a dataframe of this size, any speed difference won't be noticed.)

We built the dataframe above by using `pd.DataFrame.from_records()` on a list of dictionaries. Since `df['medals']` is another list of dictionaries, we can do the same thing again.

In [ ]:
medals_df = pd.DataFrame.from_records(df['medals'])
medals_df

Now we have two dataframes: the original `df`, and `medals_df` containing the medal columns. All we have to do is concatenate them. Since we're adding extra columns, we concatenate along the `columns` axis.

In [ ]:
df = pd.concat([df, medals_df], axis='columns')
df.head()

### Tidying up

Whichever way we did it, we ended up with the same dataframe.

We don't need the `medals` column any more (or `abbr` for that matter), so let's drop them.

In [ ]:
df = df.drop(columns=['medals', 'abbr'])
df

## Plotting
Let's get plotting, and visualise what we have.

In [ ]:
hv.Bars(df, 'name', 'total')

That was underwhelming. There are too many names to fit, and we'd have to make the plot really wide to see them all.

Let's try keeping only the names that have a total greater than zero, and making the plot wider.

In [ ]:
df = df[df['total']>0]

hv.Bars(df, 'name', 'total').opts(width=800)

That's better, but we need to fix those names.

In [ ]:
bars = hv.Bars(df, 'name', 'total').opts(width=800, xrotation=45)
bars

It's easy to find a particular country when they're in alphabetical order, but we're probably more interested in comparing countries. The bars should be sorted in order of total medals. Since Holoviews displays the dat in the order it comes, we just need to give it a sorted dataframe.

In [ ]:
bars = hv.Bars(df.sort_values('total', ascending=False), 'name', 'total').opts(width=800, xrotation=45)
bars

Another way of coping with many key values is to put them on the y axis so the bars are horizontal. The `invert_axes` option will do that. (We have to sort the values in the other direction, because Holoviews axes start at the bottom left.)

In [ ]:
hv.Bars(df.sort_values('total', ascending=True), 'name', 'total').opts(width=400, height=500, invert_axes=True)

Let's concentrate on the top ten countries. What does that mean, though? Is it top ten by medals won (which is what we've been looking at)?

In [ ]:
total_df = df.sort_values('total', ascending=False).head(10)
total_df

Or top ten by gold, then silver, then bronze medals won? We generally do this one.

In [ ]:
medal_df = df.sort_values(['gold', 'silver', 'bronze'], ascending=False).head(10).reset_index(drop=True)
medal_df

In [ ]:
medal_bars = hv.Bars(medal_df, 'name', 'total').opts(width=800)
medal_bars

Incidentally, some plotting libraries make the bars different colors by default. There's no need, because the bar totals all belong to the same dimension. However, we can do that too if we want to.

(Using `.options()` here is exactly the same as using `.opts()`, except the `medal_bars` element isn't modified.)

In [ ]:
medal_bars.options(color='name', cmap='isolum', show_legend=False)

Given the disparity between the two different top tens, we should visualise the difference. We want to see nested bars, where the main dimension is `name` and the nested dimension contains `total` and `gold`.

To do this, we need to melt the dataframe. Melting keeps "identifier" columns intact, and "unpivots" the remaining "value" columns so the column names become values. That means that the rows of the identifier columns have to be repeated for each column, so our 10 row dataframe with 4 "value" columns turns into a $10 \times 4 = 40$ row dataframe.

Here's `medal_df`.

In [ ]:
medal_df

And here's `melted_df` using the name as the "identifier" column. The medal columns have become values, and the new columns are called `variable` and `value`.

You can see why it's called `melt`: the `gold` column and values melt down next to the names, then the `total` column anf values melt down next to the names, and so on.

In [ ]:
melted_df = medal_df.melt('name')
melted_df

Let's see the result.

In [ ]:
hv.Bars(melted_df, ['name', 'variable'], 'value').opts(width=800, xrotation=90)

That's pretty much what we want, though there's some tidying up to do.
- For this comparison, we really only want to see the `total` and `gold` variables. We can remelt `medal_df` to only include those variable columns (and give the new columns better names).
- We want appropriate non-misleading colors. We can specify specific colors for `total` and `gold` using a custom colormap.

In [ ]:
melted_df = medal_df.melt('name', ['gold', 'total'], var_name='type', value_name='count')
cmap = {'gold':'gold', 'total':'Teal'}
hv.Bars(melted_df, ['name', 'type'], 'count').opts(width=800, cmap=cmap)

Another thing to look at would be the gold, silver, and bronze medals stacked to make the total. Let's remelt `medal_df` again.

We'll display the `stacked_bars` chart above the `medals_bars` chart from above so we can double-check that the stacks add up correctly. We'll also remove the lines around the bars for a sparser look.

In [ ]:
melted_df = medal_df.melt('name', ['gold', 'silver', 'bronze'], var_name='type', value_name='count')
cmap = {'gold':'gold', 'silver':'grey', 'bronze':'sandybrown'}
stacked_bars = hv.Bars(melted_df, ['name', 'type'], 'count').opts(width=800, cmap=cmap, stacked=True)

(stacked_bars + medal_bars).opts(
    hv.opts.Bars(show_grid=True, line_color=None)
).cols(1)